Trying to work with both faces and meta-data

In [1]:
import tensorflow as tf
import time
%matplotlib inline

from helpers.data_loading import *
from helpers.CNN_layers import ConvLayer, DenseLayer
from spearman import score_function

We first load the extracted faces and the meta features

In [2]:
faces, _ = load_faces(verbose=False)
faces_to_predict = faces[10000:13000]  # for final submission
faces = faces[0:10000]

In [3]:
meta, _ = load_meta_features(file="data/facial_features_train.csv")
meta_to_predict, _ = load_meta_features(file="data/facial_features_test.csv")  # for final submission

In [4]:
# here we remove the features that are only available for the train
to_keep = list(set(meta.columns).intersection(set(meta_to_predict.columns)))
to_keep.sort()  # sort in alphabetical order to always have the same order

In [5]:
meta = meta[to_keep]
meta_to_predict = meta_to_predict[to_keep]
# now we remove non-numerical features
numeric_features = np.array([type(value) is not str for value in meta.as_matrix()[0,:]])

In [6]:
meta = meta.as_matrix()[:,numeric_features]
meta_to_predict = meta_to_predict.as_matrix()[:,numeric_features]

In [7]:
meta = np.array(meta, dtype=np.float32)
meta_to_predict = np.array(meta_to_predict, dtype=np.float32)

In [8]:
y, _ = load_labels()  # scores between 0 and 25 for training instances
y = y.reshape((-1,1))

## Building a CNN with meta features connected at the dense layers

In [9]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True
sess = tf.InteractiveSession(config=config)

In [10]:
n_faces_pixels = 10**4
n_meta = meta.shape[1]
print "Face pixels:\t", n_faces_pixels
print "Meta features:\t", n_meta

Face pixels:	10000
Meta features:	93


In [11]:
input_tf = tf.placeholder(tf.float32, shape=[None, n_faces_pixels])
x_tf = tf.reshape(input_tf, [-1,100,100,1])
y_tf = tf.placeholder(tf.float32, shape=[None, 1])

In [12]:
conv1 = ConvLayer(x_tf, 32)
conv1.max_pool()

In [13]:
conv2 = ConvLayer(conv1.output, 32)
conv2.max_pool()

In [14]:
fc1 = DenseLayer(conv2.output, 1024)
fc1.drop_out()

In [15]:
fc1.output.get_shape()

TensorShape([Dimension(None), Dimension(1024)])

In [16]:
meta_tf = tf.placeholder(tf.float32, shape=[None, n_meta])
metaLayer = DenseLayer(meta_tf, 256)

In [17]:
merging_matrix = tf.concat(1, [metaLayer.output, fc1.output])
merge_layer = DenseLayer(merging_matrix, 100)

In [18]:
fc2 = DenseLayer(merge_layer.output, 1)

In [19]:
mse = tf.reduce_mean((fc2.output - y_tf)**2)
learning_rate = tf.placeholder(tf.float32, shape=[])
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mse)

In [20]:
sess.run(tf.global_variables_initializer())

In [21]:
mse.eval(feed_dict={
    input_tf:faces[0:500,:], 
    y_tf: y[0:500,:], 
    fc1.keep_prob: 1.0,
    meta_tf: meta[0:500,:]
})

177.46024

In [22]:
type(meta[0,0])

numpy.float32

In [23]:
# sample train/test indices
train = np.random.choice(range(0, 10**4), size = 9500, replace = False)
test = np.setdiff1d(np.arange(0, 10**4), train)

In [24]:
def epoch(n_ep, lr=1e-4):
    for i in range(95):
        batch_i = train[np.arange(i*100, (i+1)*100)]
        batch = (faces[batch_i,:], y[batch_i,:], meta[batch_i, :])
        train_step.run(feed_dict={
            input_tf: batch[0], 
            y_tf: batch[1], 
            meta_tf: batch[2], 
            fc1.keep_prob: 0.5,
            learning_rate: lr
        })
    train_mse = sess.run(mse, feed_dict={input_tf:batch[0], y_tf: batch[1],  meta_tf: batch[2], fc1.keep_prob: 1.0})
    print "iteration %d, training mse %g"%(n_ep, train_mse),
    print "test mse %g"%mse.eval(feed_dict={
        input_tf: faces[test,:], y_tf: y[test,:], meta_tf:meta[test, :], fc1.keep_prob: 1.0}),
    
    # compute spearman
    preds = fc2.output.eval(feed_dict={
        input_tf: faces[test,:], meta_tf:meta[test, :], fc1.keep_prob: 1.0})
    current_spearman = score_function(y_pred=np.round(preds), y_true=y[test,:])
    print "test spearman %g"%current_spearman

In [25]:
starts = time.time()
for global_it in range(100):
    epoch(global_it, 1e-4)
print "\nRunning time: ", round(time.time() - starts), "s"

iteration 0, training mse 16.9369 test mse 15.8295 test spearman 0.425988
iteration 1, training mse 15.7123 test mse 15.3016 test spearman 0.436337
iteration 2, training mse 15.7256 test mse 14.9614 test spearman 0.454195
iteration 3, training mse 15.1778 test mse 14.4667 test spearman 0.487936
iteration 4, training mse 14.9961 test mse 14.5242 test spearman 0.487189
iteration 5, training mse 14.9742 test mse 14.1519 test spearman 0.496786
iteration 6, training mse 14.6113 test mse 14.0821 test spearman 0.513367
iteration 7, training mse 14.2737 test mse 13.9818 test spearman 0.512746
iteration 8, training mse 13.7476 test mse 13.8149 test spearman 0.521635
iteration 9, training mse 13.4918 test mse 13.8698 test spearman 0.525505
iteration 10, training mse 13.0168 test mse 13.7541 test spearman 0.522929
iteration 11, training mse 12.7566 test mse 13.6132 test spearman 0.516845
iteration 12, training mse 12.4206 test mse 13.5505 test spearman 0.534069
iteration 13, training mse 11.767 t

In [26]:
starts = time.time()
for global_it in range(100, 200):
    epoch(global_it, 1e-5)  # decrease learning rate a bit
print "\nRunning time: ", round(time.time() - starts), "s"

iteration 100, training mse 0.200715 test mse 12.8359 test spearman 0.556493
iteration 101, training mse 0.195814 test mse 12.8378 test spearman 0.556708
iteration 102, training mse 0.191344 test mse 12.8343 test spearman 0.557403
iteration 103, training mse 0.185881 test mse 12.8168 test spearman 0.555992
iteration 104, training mse 0.190675 test mse 12.82 test spearman 0.556603
iteration 105, training mse 0.189832 test mse 12.8235 test spearman 0.556265
iteration 106, training mse 0.18563 test mse 12.7816 test spearman 0.558634
iteration 107, training mse 0.180877 test mse 12.7904 test spearman 0.554634
iteration 108, training mse 0.17325 test mse 12.7813 test spearman 0.555937
iteration 109, training mse 0.164084 test mse 12.7849 test spearman 0.553279
iteration 110, training mse 0.172476 test mse 12.7643 test spearman 0.554236
iteration 111, training mse 0.155743 test mse 12.7734 test spearman 0.556405
iteration 112, training mse 0.146678 test mse 12.7729 test spearman 0.555293
ite

In [27]:
for global_it in range(200, 300):
    epoch(global_it, 1e-6)  # decrease learning rate a bit

iteration 200, training mse 0.0537303 test mse 12.6724 test spearman 0.557457
iteration 201, training mse 0.0530784 test mse 12.6678 test spearman 0.558875
iteration 202, training mse 0.0518901 test mse 12.6667 test spearman 0.558143
iteration 203, training mse 0.050341 test mse 12.6687 test spearman 0.559618
iteration 204, training mse 0.0515366 test mse 12.6726 test spearman 0.560352
iteration 205, training mse 0.0525801 test mse 12.6738 test spearman 0.560927
iteration 206, training mse 0.0514248 test mse 12.673 test spearman 0.560102
iteration 207, training mse 0.0517993 test mse 12.6751 test spearman 0.558817
iteration 208, training mse 0.0511616 test mse 12.6732 test spearman 0.558796
iteration 209, training mse 0.0525301 test mse 12.6765 test spearman 0.559467
iteration 210, training mse 0.0480139 test mse 12.6761 test spearman 0.561685
iteration 211, training mse 0.0483598 test mse 12.6762 test spearman 0.560396
iteration 212, training mse 0.0497271 test mse 12.6717 test spearm

## Making a quick submission

In [162]:
preds = []
for i in range(faces_to_predict.shape[0]):
    preds.append(
        fc2.output.eval(feed_dict={
            input_tf: faces_to_predict[i:(i+1),:], meta_tf:meta_to_predict[i:(i+1),:], fc1.keep_prob: 1.0})[0]
    )

In [167]:
export_submission(preds, "CNN_faces_meta")